Cesar Notebook, do test stuff here to aid in keeping main clean

Basic Imports/ Notebook setup
it brokey

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from typing import Tuple
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import confusion_matrix, top_k_accuracy_score
import torchvision                                                       
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch import nn, optim

#setting our device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the dataset for the training/test sets
def get_data(batch_size: int):
    
    train = torchvision.datasets.FashionMNIST(root="./",train=True,transform= ToTensor())

    test_dataset = torchvision.datasets.FashionMNIST(root="./", transform = ToTensor(), train=False)    

    train_DL = DataLoader(train, batch_size=batch_size)
    test_DL = DataLoader(test_dataset, batch_size=batch_size)
    return (train_DL, test_DL)


class NeuralNetwork(nn.Module):


    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
      
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out
    


In [59]:
def subset_data(batch_size : int,subset : list):
    test_dataset = torchvision.datasets.FashionMNIST(root="./", transform = ToTensor(), train=False)
    train_sub = DataLoader(subset, batch_size=batch_size)
    test_DL = DataLoader(test_dataset, batch_size=batch_size)
    return (train_sub, test_DL)

In [42]:
def runNN(training, testing):
    model = NeuralNetwork()
    model.to(device)

    error = nn.CrossEntropyLoss()

    learning_rate = 0.001
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    
    num_epochs = 7
    count = 0
    # Lists for visualization of loss and accuracy 
    loss_list = []
    iteration_list = []
    accuracy_list = []

    # Lists for knowing classwise accuracy
    predictions_list = []
    labels_list = []
    for epoch in range(num_epochs):
        for images, labels in training:
            # Transfering images and labels to GPU if available
            images, labels = images.to(device), labels.to(device)
    
            train = Variable(images.view(100, 1, 28, 28))
            labels = Variable(labels)
            # Forward pass 
            outputs = model(train)
            loss = error(outputs, labels)
        
        # Initializing a gradient as 0 so there is no mixing of gradient among the batches
            optimizer.zero_grad()
        
        #Propagating the error backward
            loss.backward()
        
        # Optimizing the parameters
            optimizer.step()
    
            count += 1
    
         # Testing the model
    
            if not (count % 50):    # It's same as "if count % 50 == 0"
                total = 0
                correct = 0
        
                for images, labels in testing:
                    images, labels = images.to(device), labels.to(device)
                    labels_list.append(labels)
            
                    test = Variable(images.view(100, 1, 28, 28))
            
                    outputs = model(test)
            
                    predictions = torch.max(outputs, 1)[1].to(device)
                    predictions_list.append(predictions)
                    correct += (predictions == labels).sum()
            
                    total += len(labels)
            
                accuracy = correct * 100 / total
                loss_list.append(loss.data)
                iteration_list.append(count)
                accuracy_list.append(accuracy)
        
            if not (count % 500):
                print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))
                   

In [44]:
data_set = get_data(100)
runNN(data_set[0],data_set[1])

Iteration: 500, Loss: 0.5172774195671082, Accuracy: 86.56999969482422%
Iteration: 1000, Loss: 0.2936069667339325, Accuracy: 87.37000274658203%
Iteration: 1500, Loss: 0.32441645860671997, Accuracy: 88.11000061035156%
Iteration: 2000, Loss: 0.22010661661624908, Accuracy: 89.08999633789062%
Iteration: 2500, Loss: 0.1608290821313858, Accuracy: 89.54000091552734%
Iteration: 3000, Loss: 0.17620964348316193, Accuracy: 89.5%
Iteration: 3500, Loss: 0.23200824856758118, Accuracy: 91.12999725341797%
Iteration: 4000, Loss: 0.256465345621109, Accuracy: 90.58000183105469%


Data "Distilling" by that I mean subsetting

make data into list bc lol pytorch dataset class is BAD

In [50]:
#import random numpy library for subset creation
from numpy import random as rd

#load trainset data
train_set_sub = torchvision.datasets.FashionMNIST("./", transform= ToTensor())

#convert to list for easy data deletion
train_set_l = list(train_set_sub)
#2nd set, subset of deleted

In [52]:
#subset using a random uniform dist
def uniform(uniFull : list):
    deleted = []
    while len(uniFull) > 30000:
        randomVal = rd.uniform(0,1)
        a = 0
        b = len(uniFull)
        temp = randomVal
        temp2 = int(np.floor((a + (b-a)) * temp))
        deleted.append(uniFull[temp2])
        del uniFull[temp2]
    return deleted

#subset using a random accept reject
def AcRj(acrjFull: list):
    deleted = []
    while len(acrjFull) > 30000:
        del_int = rd.randint(0,60000)
        if del_int > len(acrjFull) - 1:
            while del_int > len(acrjFull) - 1:
                del_int = rd.randint(0,60000)
            deleted.append(acrjFull[del_int])
            del acrjFull[del_int]
        else:
            deleted.append(acrjFull[del_int])
            del acrjFull[del_int]
    return deleted

#subset using deleting with range
def Ac(acFull: list):
    deleted = []
    while len(acFull) > 30000:
        del_int = rd.randint(0,len(acFull))
        deleted.append(acFull[del_int])
        del acFull[del_int]
    return deleted
    

In [61]:
uniFull = train_set_l.copy()
acrjFull = train_set_l.copy()
acFull = train_set_l.copy()

delUni = uniform(uniFull)
delAcRj = AcRj(acrjFull)
delAc = Ac(acFull)
print("\nSubsets of 30000 entries\n")
print("\nSubsetted Uniform dist preformance:\n")
subsets = subset_data(100, uniFull)
runNN(subsets[0], subsets[1])


print("\nSubsetted Uniform dist preformance using deleted entries from previous model:\n")
subset = subset_data(100, delUni)
runNN(subset[0], subset[1])


print("\nSubsetted Accept Reject preformance:\n")
subset = subset_data(100, acrjFull)
runNN(subset[0], subset[1])



print("\nSubsetted Accept Reject preformance using deleted entries from previous model:\n")
subset = subset_data(100, delAcRj)
runNN(subset[0], subset[1])



print("\nSubsetted Accept preformance:\n")
subset = subset_data(100, acFull)
runNN(subset[0], subset[1])



print("\nSubsetted Accept preformance using deleted entries from previous model:\n")
subset = subset_data(100, delAc)
runNN(subset[0], subset[1])





Subsets of 30000 entries


Subsetted Uniform dist preformance:

Iteration: 500, Loss: 0.4203479290008545, Accuracy: 87.83000183105469%
Iteration: 1000, Loss: 0.34003502130508423, Accuracy: 86.5999984741211%
Iteration: 1500, Loss: 0.11586302518844604, Accuracy: 88.91000366210938%
Iteration: 2000, Loss: 0.2227170765399933, Accuracy: 87.29000091552734%

Subsetted Uniform dist preformance using deleted entries from previous model:

Iteration: 500, Loss: 0.5053504109382629, Accuracy: 87.11000061035156%
Iteration: 1000, Loss: 0.3504447937011719, Accuracy: 86.88999938964844%
Iteration: 1500, Loss: 0.24270737171173096, Accuracy: 88.73999786376953%
Iteration: 2000, Loss: 0.27707022428512573, Accuracy: 88.80000305175781%

Subsetted Accept Reject preformance:

Iteration: 500, Loss: 0.29580649733543396, Accuracy: 87.02999877929688%
Iteration: 1000, Loss: 0.28943580389022827, Accuracy: 89.04000091552734%
Iteration: 1500, Loss: 0.19460120797157288, Accuracy: 88.47000122070312%
Iteration: 2000, Loss